<a href="https://colab.research.google.com/github/ChenHaaa/Algorithm1_2025/blob/25_2_s_14/ex7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# All-in-one solutions + demonstrations for Problem Set #7
# Run in Colab / local Python (requires only built-in libs)

from collections import defaultdict
from itertools import combinations

# ---------------------------
# Helpers: display utilities
# ---------------------------
def show(title):
    print("\n" + "="*60)
    print(title)
    print("="*60)

# ---------------------------
# Part 1: Directed graphs and their transposes
# ---------------------------
def transpose_directed_graph(adj):
    """adj: dict node -> list of neighbors (directed edges u->v)"""
    t = {u: [] for u in adj}
    for u, nbrs in adj.items():
        for v in nbrs:
            if v not in t:
                t[v] = []
            t[v].append(u)
    return t

show("Part 1 — Directed graphs and transposes")

# Example A: simple chain
G_A = {1: [2], 2: [3], 3: []}
print("G_A:", G_A)
print("G_A^T:", transpose_directed_graph(G_A))

# Example B: 3-cycle
G_B = {"A":["B"], "B":["C"], "C":["A"]}
print("\nG_B (3-cycle):", G_B)
print("G_B^T (should be same as G_B):", transpose_directed_graph(G_B))

# Example C: mixed directions
G_C = {"u":["v","w"], "v":[], "w":["v"]}
print("\nG_C:", G_C)
print("G_C^T:", transpose_directed_graph(G_C))

# ---------------------------
# Part 2: Undirected graphs and their complements (inverse)
# ---------------------------
def complement_undirected_graph(nodes, edges):
    """nodes: iterable of nodes
       edges: set of frozenset pairs {u,v}
       returns complement edges set (no self-loops)
    """
    node_list = list(nodes)
    all_pairs = {frozenset({u,v}) for u,v in combinations(node_list, 2)}
    comp = all_pairs - set(edges)
    return comp

show("Part 2 — Undirected graphs and complements")

# Example U1
nodes = ["A","B","C","D"]
edges_U1 = {frozenset({"A","B"}), frozenset({"B","C"})}
print("G (U1) edges:", edges_U1)
comp_U1 = complement_undirected_graph(nodes, edges_U1)
print("Complement of U1:", comp_U1)

# Example U2 (complete graph K4)
edges_K4 = {frozenset(e) for e in combinations(nodes,2)}
print("\nG = K4 edges (6 edges):", edges_K4)
print("Complement of K4 (should be empty):", complement_undirected_graph(nodes, edges_K4))

# Example U3 (empty graph)
edges_empty = set()
print("\nG = empty graph edges:", edges_empty)
print("Complement (should be K4):", complement_undirected_graph(nodes, edges_empty))

# ---------------------------
# Part 3: If original is dense -> complement is sparse
# ---------------------------
show("Part 3 — Dense vs Complement")
n = 8
# create nearly complete graph with a few missing edges
nodes_n = list(range(n))
complete_edges = {frozenset(e) for e in combinations(nodes_n,2)}
# remove 3 edges to make it "nearly complete"
dense_edges = set(complete_edges)
to_remove = list(dense_edges)[:3]
for r in to_remove:
    dense_edges.remove(r)
comp_dense = complement_undirected_graph(nodes_n, dense_edges)
print(f"Original edges count: {len(dense_edges)} (nearly complete, max {len(complete_edges)})")
print(f"Complement edges count: {len(comp_dense)} (should be small)")

# ---------------------------
# Part 4: Dual graphs for simple planar examples
# ---------------------------
# We'll build duals from an explicit planar embedding (face list).
# For small examples we give faces manually.
def build_dual_from_faces(edges, faces):
    """
    edges: set of frozenset({u,v}) for original graph
    faces: list of faces; each face is set of edges (frozensets) that bound that face
           faces should include the outer face as one of the faces.
    returns: dual as adjacency multimap mapping face_index -> list of adjacent face_indices.
             We return parallel edges multiple times (list may contain duplicates).
    """
    # map edge -> list of face indices that contain that edge
    edge_to_faces = defaultdict(list)
    for fi, face in enumerate(faces):
        for e in face:
            edge_to_faces[e].append(fi)
    # build dual adjacency (parallel edges possible)
    dual = defaultdict(list)
    for e, flist in edge_to_faces.items():
        # every pair of faces that share this edge becomes an edge in the dual
        for i in range(len(flist)):
            for j in range(i+1, len(flist)):
                f1, f2 = flist[i], flist[j]
                dual[f1].append(f2)
                dual[f2].append(f1)
    return dict(dual)

show("Part 4 — Dual graphs (explicit faces)")

# Example D1: triangle
# Nodes: A,B,C
edges_tri = {frozenset({"A","B"}), frozenset({"B","C"}), frozenset({"C","A"})}
# faces: inner face uses all 3 edges; outer face also bounded by same 3 edges
faces_tri = [set(edges_tri), set(edges_tri)]  # face 0 = interior, face 1 = exterior
dual_tri = build_dual_from_faces(edges_tri, faces_tri)
print("Triangle edges:", edges_tri)
print("Triangle faces (inner, outer): each uses same 3 edges")
print("Dual adjacency (face indices):", dual_tri)
print("Interpretation: two dual vertices connected by 3 parallel edges (multi-edges).")

# Example D2: square with diagonal (1-2-3-4-1 and diagonal 1-3)
edges_sq_diag = {frozenset(e) for e in [("1","2"),("2","3"),("3","4"),("4","1"),("1","3")]}
# faces (explicit planar embedding): two triangular faces and outer face.
# face A: triangle 1-2-3 -> edges (1-2,2-3,1-3)
# face B: triangle 1-3-4 -> edges (1-3,3-4,4-1)
# outer face: edges (1-2,2-3,3-4,4-1) but note diagonal is interior
faceA = {frozenset(("1","2")), frozenset(("2","3")), frozenset(("1","3"))}
faceB = {frozenset(("1","3")), frozenset(("3","4")), frozenset(("4","1"))}
faceOut = {frozenset(("1","2")), frozenset(("2","3")), frozenset(("3","4")), frozenset(("4","1"))}
faces_sq = [faceA, faceB, faceOut]
dual_sq = build_dual_from_faces(edges_sq_diag, faces_sq)
print("\nSquare+diagonal edges:", edges_sq_diag)
print("Faces (A,B,outer):")
print(" Dual adjacency:", dual_sq)
print("Interpretation: 3 dual vertices forming a triangle (each face adjacent to the two others).")

# ---------------------------
# Part 5: Why dual only for planar graphs / non-planar example
# ---------------------------
show("Part 5 — Duals and non-planar graphs")

explanation = """
The dual is defined by placing one vertex in each face of a planar embedding and connecting
two dual vertices by an edge for each primal edge that separates the corresponding two faces.
If a graph is non-planar (no planar embedding exists), there are no well-defined faces in the plane,
so you cannot construct the planar dual.

Classic non-planar graphs: K5 and K3,3 (K_{3,3}).
K3,3 cannot be drawn without crossings; therefore there is no planar dual for it.
"""
print(explanation)

# Show K3,3 adjacency and reason briefly
k33_nodes = {"L1","L2","L3","R1","R2","R3"}
k33_edges = {frozenset({l,r}) for l in ["L1","L2","L3"] for r in ["R1","R2","R3"]}
print("K3,3 edges count:", len(k33_edges))
print("K3,3 is non-planar (Kuratowski): has no planar embedding -> no dual.")

# ---------------------------
# Part 6: Bron–Kerbosch (no pivot) implementation with full trace
# ---------------------------
show("Part 6 — Bron-Kerbosch tracing and maximal cliques")

def bron_kerbosch_trace(graph):
    """
    graph: dict node -> set(neigh)
    prints trace of calls and returns list of maximal cliques
    """
    maximal_cliques = []
    call_id = 0
    def bk(R, P, X, depth=0):
        nonlocal call_id
        cid = call_id
        call_id += 1
        indent = "  " * depth
        print(f"{indent}Call {cid}: R={sorted(R)}, P={sorted(P)}, X={sorted(X)}")
        if not P and not X:
            print(f"{indent}  -> Report clique: {sorted(R)}")
            maximal_cliques.append(set(R))
            return
        # iterate over a snapshot of P
        for v in list(P):
            newR = set(R); newR.add(v)
            newP = P.intersection(graph[v])
            newX = X.intersection(graph[v])
            print(f"{indent}  Recurse on v={v}")
            bk(newR, newP, newX, depth+1)
            P.remove(v)
            X.add(v)
            print(f"{indent}  After backtrack v={v}: P now={sorted(P)}, X now={sorted(X)}")
    nodes = set(graph.keys())
    bk(set(), set(nodes), set(), depth=0)
    return maximal_cliques

# Graph from Problem 2
graphG = {
    "A": set(["B","C"]),
    "B": set(["A","C"]),
    "C": set(["A","B","D"]),
    "D": set(["C"])
}

print("Graph G adjacency:", graphG)
cliques = bron_kerbosch_trace(graphG)
print("\nAll maximal cliques found:", cliques)
# Identify maximum cliques (largest by size)
max_size = max(len(c) for c in cliques)
max_cliques = [c for c in cliques if len(c)==max_size]
print("Maximum clique size:", max_size)
print("Maximum clique(s):", max_cliques)

# ---------------------------
# End
# ---------------------------
print("\n\nDone. The script demonstrated all requested items (see printed output).")



Part 1 — Directed graphs and transposes
G_A: {1: [2], 2: [3], 3: []}
G_A^T: {1: [], 2: [1], 3: [2]}

G_B (3-cycle): {'A': ['B'], 'B': ['C'], 'C': ['A']}
G_B^T (should be same as G_B): {'A': ['C'], 'B': ['A'], 'C': ['B']}

G_C: {'u': ['v', 'w'], 'v': [], 'w': ['v']}
G_C^T: {'u': [], 'v': ['u', 'w'], 'w': ['u']}

Part 2 — Undirected graphs and complements
G (U1) edges: {frozenset({'B', 'A'}), frozenset({'B', 'C'})}
Complement of U1: {frozenset({'D', 'C'}), frozenset({'C', 'A'}), frozenset({'D', 'A'}), frozenset({'D', 'B'})}

G = K4 edges (6 edges): {frozenset({'B', 'A'}), frozenset({'D', 'B'}), frozenset({'B', 'C'}), frozenset({'D', 'A'}), frozenset({'D', 'C'}), frozenset({'C', 'A'})}
Complement of K4 (should be empty): set()

G = empty graph edges: set()
Complement (should be K4): {frozenset({'B', 'A'}), frozenset({'D', 'A'}), frozenset({'D', 'B'}), frozenset({'D', 'C'}), frozenset({'B', 'C'}), frozenset({'C', 'A'})}

Part 3 — Dense vs Complement
Original edges count: 25 (nearly comple